![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [22]:
# dataset
PATH = "/content/arabic words excel (2).xlsx"

In [5]:
!pip install pyarabic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 3.8 MB/s eta 0:00:00


In [6]:
def preprocessing(sentence):
    # Add space after colon
    sentence = sentence.replace(':', ' : ')
    # Remove extra space between words
    sentence = ' '.join(sentence.split())
    # Remove Diacritics
    sentence = araby.strip_diacritics(sentence)
    # Replace period wiht new line
    sentence = sentence.replace('.', '\n')
    # Remove extra new lines after the last word
    sentence = sentence.rstrip()
    return sentence




In [23]:
import pandas as pd
import numpy as np
import pyarabic.araby as araby

np.random.seed(42)

df = pd.read_excel(PATH)

# Shuffle the dataframe
df = df.sample(frac=1).reset_index(drop=True)

# Data preprocessing
df['الإعراب'] = df['الإعراب'].apply(lambda x: preprocessing(x))
df['الجملة'] = df['الجملة'].apply(lambda x: preprocessing(x))

df.head()

,الجملة,الإعراب,المصدر
0,قال الطالب : أنا ناجح,قال : فعل ماضي\nالطالب : فاعل مرفوع وعلامة رفع...,NaN
1,المرضى نائمون,المرضى : مبتدأ مرفوع وعلامة رفعه الضمة المقدرة...,NaN
2,الطائر يطير في السماء,الطائر : مبتدأ مرفوع وعلامة رفعه الضمة الظاهرة...,NaN
3,الذي يفوز فله جائزة,الذي : اسم موصول متضمن معنى الشرط،مبني على الس...,NaN
4,إن العلم طريقه طويلة,إن : حرف توكيد ونصب مبني على الفتحة لا محل له ...,NaN


# Generate wrong samples ( word level )


In [24]:
import random

words = [sentence.split('\n') for sentence in list(df['الإعراب'])]
sen_words = {sen:words for sen,words in zip(list(df['الجملة']),words)}
# words

In [25]:
random.seed(42)


def generate_random_irab(word):


    roles = [
    "اسم موصول", "مفعول فيه", "ظرف زمان", "مبتدأ", "خبر", "فاعل", "مفعول به",
    "حال", "تمييز", "مضاف إليه", "نعت", "عطف بيان", "توكيد", "بدل", "مفعول مطلق",
    "مفعول لأجله", "مفعول معه", "مستثنى", "منادى", "اسم إن", "خبر إن", "اسم كان",
    "خبر كان", "نائب فاعل", "حرف جر", "حرف توكيد ونصب", "حرف عطف","اسم مجرور"
    ]

    cases = ["مرفوع", "منصوب", "مجرور", "مجزوم", "مبني"]

    case_markers = {
        "مرفوع": ["الضمة", "الواو", "الألف", "ثبوت النون"],
        "منصوب": ["الفتحة", "الألف", "الياء", "الكسرة", "حذف النون"],
        "مجرور": ["الكسرة", "الياء", "الفتحة"],
        "مجزوم": ["السكون", "حذف حرف العلة", "حذف النون"]
    }

    results = []
    role = random.choice(roles)
    case = random.choice(cases)

    if case == "مبني":
        case_description = f"مبني على {random.choice(['الضم', 'الفتح', 'الكسر', 'السكون'])}"
    else:
        # Based on the case, choose a random case marker that could come with this case
        marker = random.choice(case_markers.get(case, ["علامة غير محددة"]))

        # I think this is clear no need for comments
        case_marker = {
            "مرفوع": "رفعه",
            "منصوب": "نصبه",
            "مجرور": "جره",
            "مجزوم": "جزمه"
        }.get(case, "إعرابه")
        case_description = f"{case} وعلامة {case_marker} {marker}"

    irab = f"{word} : {role} {case_description}"

    # Remove repeated words (just to make sure that the case اسم مجرور مجرور will be اسم مجرور)
    irab = irab.split()
    irab = list(dict.fromkeys(irab))
    irab = ' '.join(irab)
    return irab

wrong_sen_words = {}
for sen,words in sen_words.items():
    wrong_words = []
    for word in words:
        word = word.split(' : ')[0]
        wrong_words.append(generate_random_irab(word))
    wrong_sen_words[sen] = wrong_words

# df['اعراب خاطئ'] = wrong_examples
# df['اعراب صحيح'] = words
# df.head()
sen_words[sen]

['دائما : مفعول فيه ظرف زمان، متعلق بالفعل أحب',
 'أحب : فعل مضارع مرفوع وعلامة رفعه الضمة الظاهرة، والفاعل ضمير مستتر فيه وجوبا تقديره "أنا"',
 'والدي : مفعول به منصوب وعلامة نصبه الفتحة المقدرة على ما قبل الياء منع من ظهورها اشتغال المحل بالحركة المناسبة للياء،وهومضاف،الياء : ضميرمتصل مبني على السكون في محل جر مضاف اليه']

# API key

In [8]:
# with open('examples.txt','w') as f:
#         f.write(examples[0])

In [ ]:
import os
import getpass
key = ''
def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
		"apikey" : getpass.getpass(f"{key}")
	}


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [10]:
model_id = "sdaia/allam-1-13b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [11]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 900,
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = ''
space_id = None


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [13]:
from IPython.display import clear_output
!pip install ibm_watsonx_ai
clear_output()

In [14]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)


vGt-8Abv5RGFaVdJ8OvBibAMOCZCMp1Vgg8kdkn0zbtd··········


/usr/local/lib/python3.10/dist-packages/ibm_watsonx_ai/foundation_models/model.py:101: DeprecationWarning: The `Model` class is deprecated and will be removed in a future release. Please use the `ModelInference` class instead. To update your imports, use: `from ibm_watsonx_ai.foundation_models import ModelInference`.
  warn(


## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


1473


In [35]:
def prompt_name(name):
  few_shots = """
<s> [INST]
        الجمله: ما أهملت واجبى قط
        الاعراب: ما : اسم إن منصوب وعلامة نصبه الألف
 [/INST]
 خاطئ
الإعراب الصحيح: ما: اسم نكرة مبني على السكون في محل رفع مبتدأ
</s><s> [INST]
الجمله: الطائر يطير في السماء
الاعراب : يطير : فعل مضارع مرفوع وعلامة رفعه الضمة الظاهرة على آخره،والفاعل ضمير مستتر تقديره "هو" يعود على الطائر
[/INST]
صحيح
</s><s> [INST]
الجمله: الذي يفوز فله جائزة
الاعراب: يفوز : فعل مضارع مرفوع،وعلامة رفعه الضمة الظاهرة على آخره
[/INST]
صحيح
</s><s> [INST]
الجمله: كل من تلقاه يشكو دهره
الاعراب: يشكو : فعل مضارع مرفوع وعلامة رفعه الضمة المقدرة على الواو للثقل،والفاعل ضمير مستتر تقديره "هو"
 [/INST]
صحيح
</s><s> [INST]
الجمله: خرج القطار من المحطة
الاعراب: المحطة : مفعول لأجله مرفوع وعلامة رفعه الضمة
[/INST]
خاطئ
الاعراب الصحيح: المحطة: مفعول به منصوب وعلامة نصبه الفتحة
</s><s> [INST]
الجمله: دائما أحب والدى
الاعراب: والدي : توكيد منصوب وعلامة نصبه الكسرة
[/INST]
خاطئ
الإعراب الصحيح: والدي: مفعول به منصوب وعلامة نصبه الفتحة
</s><s> [INST]
الجمله: دائما أحب والدى
الإعراب : أحب : خبر كان منصوب وعلامة نصبه حذف النون
[/INST]
خاطئ
الإعراب الصحيح:أحب: فعل مضارع مرفوع وعلامة رفعه الضمة.
</s><s> [INST]
الجمله: دائما أحب والدى
الإعراب: دائما : مفعول مطلق مجرور وعلامة جره الكسرة
[/INST]
خاطئ
الإعراب الصحيح: دائما: ظرف زمان منصوب وعلامة نصبه الفتحة
</s><s> [INST]
الجمله: الأولاد يلعبون في الحديقة
الاعراب: الحديقة : حرف جر مبني على السكون
[/INST]
خاطئ
الإعراب الصحيح: الحديقة : اسم مجرور وعلامة جره الكسرة الظاهرة على آخره
<s></s> [INST]
الجمله: الأولاد يلعبون في الحديقة
الاعراب: في : عطف بيان مجرور وعلامة جره الفتحة
[/INST]
خاطئ
الإعراب الصحيح: حرف جر
</s>"""

  zero_shots = """
  حدد اذا كان الاعراب المعطى للكلمة صحيحه ام خاطئة, فقط اجب بصحيح او خاطئ
  اذا كان الاعراب المعطى خاطئ, اجب ب خاطئ مع تصحيح الاعراب
  """
  cot = """
  <s> [INST]
        الجمله: ما أهملت واجبى قط
        الاعراب: ما : اسم إن منصوب وعلامة نصبه الألف
 لنقم بتحليل الجملة خطوة بخطوة: [/INST]

 1- تحديد نوع الجملة: الجملة المعطاة هي جملة اسمية، لأن الفاعل (ما) ليس اسماً لشخص أو مكان أو زمان، بل هو اسم يفيد العموم والاستغراق (اسم نكرة).

2- تحديد وظيفة الكلمة: في هذه الجملة، الكلمة الرئيسية هي "ما" وتعني النفي.

3- تحديد إعرابها: بناءً على نوع الجملة ووظيفة الكلمة، يمكن تحديد إعراب "ما" كالتالي:
   - ما: اسم نكرة مبني على السكون (اسم نكرة يفيد العموم والاستغراق) في محل رفع مبتدأ.

4- مقارنة الإعراب المعطى مع الإعراب الصحيح:
   - الإعراب المعطى: ما (اسم إن منصوب وعلامة نصبه الألف)
   - الإعراب الصحيح: ما (اسم نكرة مبني على السكون في محل رفع مبتدأ)

الإجابة النهائية: خاطئ

</s>
<s>[INST]
        الجمله: الطائر يطير في السماء
        الاعراب: يطير : فعل مضارع مرفوع وعلامة رفعه الضمة الظاهرة على آخره،والفاعل ضمير مستتر تقديره "هو" يعود على الطائر
 لنقم بتحليل الجملة خطوة بخطوة: [/INST]
 1- تحديد نوع الجملة: الجملة المعطاة هي جملة فعلية لأن الفعل الرئيسي فيها هو "يطير" ويتحدث عن حدوث الفعل.

2- تحديد وظيفة الكلمة:
   - الطائر: اسم مرفوع وهو فاعل الفعل "يطير".
   - يطير: فعل مضارع مرفوع.
   - في: حرف جر.
   - السماء: اسم مجرور وعلامة جره الكسرة الظاهرة على آخره.

3- تحديد إعراب الكلمات:
   - الطائر: فاعل مرفوع وعلامة رفعه الضمة الظاهرة على آخره.
   - يطير: فعل مضارع مرفوع وعلامة رفعه الضمة الظاهرة على آخره، والفاعل ضمير مستتر تقديره "هو".
   - في: حرف جر.
   - السماء: اسم مجرور وعلامة جره الكسرة الظاهرة على آخره.

4- مقارنة الإعراب المعطى مع الإعراب الصحيح:
   - الإعراب المعطى يتطابق مع الإعراب الصحيح الذي تم تحديده في الخطوة الثالثة.

الإجابة النهائية: صحيح
</s><s>[INST]
        الجمله: الذي يفوز فله جائزة
        الاعراب: يفوز : فعل مضارع مرفوع،وعلامة رفعه الضمة الظاهرة على آخره
 لنقم بتحليل الجملة خطوة بخطوة: [/INST]
 1- تحديد نوع الجملة: الجملة المعطاة تبدو جملة فعلية لأنها تحتوي على فعل (يفوز).

2- تحديد وظيفة الكلمة: في هذه الجملة، الفعل الرئيسي هو "يفوز" والفاعل المستتر تقديره "هو".

3- تحديد إعراب الكلمة:
   - يفوز: فعل مضارع مرفوع، وعلامة رفعه الضمة الظاهرة على آخره. (هذا الإعراب المعطى)

4- مقارنة الإعراب المعطى مع الإعراب الصحيح:
   - الإعراب المعطى: يفوز (فعل مضارع مرفوع، وعلامة رفعه الضمة الظاهرة على آخره)
   - الإعراب الصحيح: يفوز (فعل مضارع مرفوع، وعلامة رفعه الضمة الظاهرة على آخره)

بناءً على المقارنة، يتطابق الإعراب المعطى مع الإعراب الصحيح.

الإجابة النهائية: صحيح
</s><s>[INST]
        الجمله: كل من تلقاه يشكو دهره
        الاعراب: يشكو : فعل مضارع مرفوع وعلامة رفعه الضمة المقدرة على الواو للثقل،والفاعل ضمير مستتر تقديره "هو"
 لنقم بتحليل الجملة خطوة بخطوة: [/INST]
 1- تحديد نوع الجملة: الجملة المعطاة تبدو جملة فعلية لأن الفعل الظاهر فيها هو "يشكو".

2- تحديد وظيفة الكلمة: الكلمة المعنية هي "يشكو" وهي فعل مضارع.

3- تحديد إعرابها:
   - يشكو: فعل مضارع مرفوع وعلامة رفعه الضمة المقدرة على الواو للثقل.
   - الفاعل: ضمير مستتر تقديره "هو".

4- مقارنة الإعراب المعطى مع الإعراب الصحيح:
   - الإعراب المعطى يتطابق مع الإعراب الصحيح للفعل المضارع "يشكو" وفاعله المستتر.

الإجابة النهائية: صحيح
</s><s>[INST]
        الجمله: خرج القطار من المحطة
        الاعراب: المحطة : مفعول لأجله مرفوع وعلامة رفعه الضمة
 لنقم بتحليل الجملة خطوة بخطوة: [/INST]
 1- تحديد نوع الجملة: الجملة المعطاة هي جملة فعلية، لأنها تبدأ بالفعل خرج (قطار).

2- تحديد وظيفة الكلمة: في هذه الجملة، الكلمة الرئيسية هي "القطار" وهو الفاعل الذي يقوم بالفعل. أما "المحطة" فهي المفعول به الذي يحدث له الفعل.

3- تحديد إعراب الكلمة: في هذه الحالة، يجب أن يكون إعراب "المحطة" هو المفعول به لا المفعول لأجله. المفعول لأجله يأتي لبيان سبب حدوث الفعل، وفي الجملة المعطاة، لا يوجد سبب واضح لخروج القطار من المحطة (مثل الهروب من شيء أو الوصول إلى مكان معين).

4- مقارنة الإعراب المعطى مع الإعراب الصحيح:
   - الإعراب المعطى: المحطة: مفعول لأجله مرفوع وعلامة رفعه الضمة.
   - الإعراب الصحيح: المحطة: مفعول به منصوب وعلامة نصبه الفتحة.

بناءً على التحليل، الإجابة النهائية هي: خاطئ.
</s><s> [INST]
        الجمله: دائما أحب والدى
        الاعراب: والدي : توكيد منصوب وعلامة نصبه الكسرة
 لنقم بتحليل الجملة خطوة بخطوة: [/INST]
 1- تحديد نوع الجملة: الجملة المعطاة هي جملة اسمية، لأن الفاعل (والدي) اسم والفعل (أحب) ليس دالا على حدث يقع في زمن محدد.

2- تحديد وظيفة الكلمة: في هذه الجملة، "والدي" هو الفاعل المتصل بالفعل "أحب" ويعمل كمفعول به أول للفعل.

3- تحديد إعرابها: بناءً على الوظيفة المذكورة أعلاه، يجب أن يكون إعراب "والدي" هو المفعول به الأول (المنصوب) للفعل "أحب".

4- مقارنة الإعراب المعطى مع الإعراب الصحيح:

الإعراب المعطى: والدي: توكيد منصوب وعلامة نصبه الكسرة.
الإعراب الصحيح: والدي: مفعول به منصوب وعلامة نصبه الفتحة.

الإجابة النهائية: خاطئ
</s><s> [INST]
        الجمله: دائما أحب والدى
        الاعراب: أحب : خبر كان منصوب وعلامة نصبه حذف النون
 لنقم بتحليل الجملة خطوة بخطوة: [/INST]
 1- تحديد نوع الجملة: الجملة المعطاة تبدو جملة فعلية لأنها تبدأ بالفعل "أحب". لكن نحتاج إلى معرفة الفاعل والمفعول به لتحديد نوعها بدقة.
2- تحديد وظيفة الكلمة: في هذه الجملة، الفاعل غير واضح (مفقود)، لكن يمكننا تحديد المفعول به وهو "والدي".
3- تحديد إعراب الكلمة:
   أ- "دائماً": ظرف زمان منصوب وعلامة نصبه الفتحة.
   ب- "أحب": فعل مضارع مرفوع وعلامة رفعه الضمة.
   ج- "والدي": مفعول به منصوب وعلامة نصبه الفتحة المقدرة لأنه اسم مقصور. والياء ضمير متصل في محل جر مضاف إليه.
4- مقارنة الإعراب المعطى مع الإعراب الصحيح:
   الإعراب المعطى يقول إن "أحب" هو خبر كان منصوب وعلامة نصبه حذف النون. لكن الجملة المعطاة لا تحتوي على كان ولا يوجد حذف لنون.
التصحيح: الجملة الصحيحة يجب أن تكون "دائماً أحب والدي" بدون ذكر "كان" وحذف النون. وإعراب "أحب" في هذه الجملة هو فعل مضارع مرفوع وعلامة رفعه الضمة الظاهرة على آخره.
الإجابة النهائية: خاطئ
</s><s> [INST]
        الجمله: دائما أحب والدى
        الاعراب: دائما : مفعول مطلق مجرور وعلامة جره الكسرة
 لنقم بتحليل الجملة خطوة بخطوة: [/INST]
 1- تحديد نوع الجملة: الجملة المعطاة هي جملة فعلية، لأنها تبدأ بالفعل "أحب".
2- تحديد وظيفة الكلمة:
   دائما: ظرف زمان منصوب وعلامة نصبه الفتحة.
   أحب: فعل مضارع مرفوع وعلامة رفعه الضمة.
   والدي: فاعل مرفوع وعلامة رفعه الضمة المقدرة وهو مضاف. والياء ضمير متصل في محل جر مضاف إليه.
3- تحديد إعرابها:
   دائما: ظرف زمان منصوب .
   أحب: فعل مضارع مرفوع.
   والدي: فاعل مرفوع.
4- مقارنة الإعراب المعطى مع الإعراب الصحيح:
   الإعراب المعطى: دائما: مفعول مطلق مجرور وعلامة جره الكسرة.
   الإعراب الصحيح: دائما: ظرف زمان منصوب وعلامة نصبه الفتحة.
بناءً على التحليل أعلاه، الإجابة النهائية هي: خاطئ.
</s><s> [INST]
        الجمله: الأولاد يلعبون في الحديقة
        الاعراب: الحديقة : حرف جر مبني على السكون
 لنقم بتحليل الجملة خطوة بخطوة: [/INST]
 1- تحديد نوع الجملة: الجملة المعطاة هي جملة فعلية، لأنها تبدأ بالفعل (يلعبون). الفعل الرئيسي هو "يلعبون" والجملة تصف حالة الأولاد وهم يلعبون.

2- تحديد وظيفة الكلمة: في هذه الجملة، الكلمة التي تحتاج إلى تحليل إعرابها هي "الحديقة". تعتبر الحديقة في هذه الجملة مفعولاً فيه (المكان الذي يجري فيه الفعل).

3- تحديد إعراب الكلمة: في الجملة "الأولاد يلعبون في الحديقة"، إعراب "الحديقة" هو:
   - الحديقة: اسم مجرور وعلامة جره الكسرة الظاهرة على آخره.

4- مقارنة الإعراب المعطى مع الإعراب الصحيح:
   - الإعراب المعطى: الحديقة : حرف جر مبني على السكون
   - الإعراب الصحيح: الحديقة : اسم مجرور وعلامة جره الكسرة الظاهرة على آخره

بناءً على المقارنة، نجد أن الإعراب المعطى غير صحيح. الإجابة النهائية: خاطئ.
</s><s> [INST]
        الجمله: الأولاد يلعبون في الحديقة
        الاعراب: في : عطف بيان مجرور وعلامة جره الفتحة
 لنقم بتحليل الجملة خطوة بخطوة: [/INST]
 1- تحديد نوع الجملة: الجملة المعطاة هي جملة فعلية، لأنها تبدأ بالفعل (الأولاد يلعبون).

2- تحديد وظيفة الكلمة: في هذه الجملة، الكلمة المطلوب تحليلها هي "في". تعتبر "في" حرف جر.

3- تحديد إعراب "في":
   - حرف جر (في هذا السياق) يعرب حرفاً مبنياً ولا محل له من الإعراب.

4- مقارنة الإعراب المعطى مع الإعراب الصحيح:
   - الإعراب المعطى يقول إن "في" هي عطف بيان مجرور وعلامة جره الفتحة.
   - الإعراب الصحيح لـ"في" هو حرف جر مبني على السكون ولا محل له من الإعراب.

الإجابة النهائية: خاطئ
</s>
  """

  dic = {'cot':cot,"zero_shots":zero_shots,"few_shots":few_shots,'zero_cot':zero_shots}
  return dic[name]


In [36]:
all_prompts = ['cot',"zero_shots","few_shots",'zero_cot']



## Evaluation


In [37]:
questions = []

for sen,words in sen_words.items():
    for word in words:
        question = f"""
        الجمله: {sen}
        الاعراب: {word}"""
        questions.append(question)

for sen,words in wrong_sen_words.items():
    for word in words:
        question = f"""
        الجمله: {sen}
        الاعراب: {word}"""
        questions.append(question)

gt = [1]*(int(len(questions)/2)) +  [0]*(int(len(questions)/2))

In [21]:
!pip install wandb


In [ ]:

import wandb

# Replace with your actual API key
wandb_api_key = ""

wandb.login(key=wandb_api_key)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [20]:
def accuracy(ground_truth, predections):
    return float((np.array(ground_truth)==np.array(predections)).sum()/len(ground_truth))



In [ ]:
import time
for name in all_prompts:
  preds = []
  prompt = prompt_name(name)

  wandb.init(
      # set the wandb project where this run will be logged
      project=name,

  )
  wandb.log({'prompt':prompt})

  all_times = 0
  for i,q in enumerate(questions):
      question = f"""{prompt}<s> [INST]
      {q} [/INST]"""
      if 'cot' in name:
        print('x')
        question = f"""<s> [INST] {prompt}
        {q}
        لنقم بتحليل الجملة خطوة بخطوة:
        [/INST]"""
      start = time.time()
      generated_response = model.generate_text(prompt=question, guardrails=False)
      end = time.time()
      wandb.log({"response_time": end-start})
      all_times += end-start
      if "خاطئ" in generated_response or "غير صحيح" in generated_response:
          preds.append(0)
      elif "صحيح" in generated_response:
          preds.append(1)
      else:
          preds.append(2)
      if i%50 == 0:
        print(generated_response)
      print(f'{i+1}/{len(questions)}')
      if i == 1:
        print(f' question: {question}')
      print(preds[-1])

  wandb.log({"response_time_AVG": all_times/len(questions)})
  print(f'{name} response_time_AVG: {all_times/len(questions)}')
  wandb.log({'accuracy': {accuracy(gt,preds)}})
  print(f'{name} accuracy: {accuracy(gt,preds)}')

  wandb.finish()


response_time,▁▂▁▁▄▃▄▆▅▂▂▄▂▁▃▁▅▃▅▃█▂▃▄▂▃▄▁▂▁▄▄▅▂▃▃▂▃▂▁
prompt,[INST]<> ...
response_time,6.2275


x
T]
 1- تحديد نوع الجملة: الجملة المعطاة هي جملة فعلية، لأنها تبدأ بالفعل "قال".

2- تحديد وظيفة الكلمة: في هذه الجملة، الفعل الرئيسي هو "قال" ويعبر عن حدث ماضي.

3- تحديد إعراب الكلمة:
   - قال: فعل ماضي مبني على الفتح.

4- مقارنة الإعراب المعطى مع الإعراب الصحيح:
   - الإعراب المعطى يقول إن "قال" هو فعل ماضي.
   - الإعراب الصحيح لـ"قال" هو فعل ماضي مبني على الفتح.

الإجابة النهائية: صحيح

في الجملة "قال الطالب: أنا ناجح"، الفعل "قال" هو فعل ماضي ويعرب على أنه فعل ماضي مبني على الفتح. 
1/744
1
x
T]
2/744
 question: <s> [INST] <s> [INST]<<SYS>>

  انت بروفيسور خبير في اللغة العربية وتستطيع اعراب اي جملة بشكل صحيح و دقيق

    المهمة: يجب عليك الان ان تتحقق مما إذا كان إعراب كلمة في جملة عربية صحيحًا.
    انت تتبع خطوات منطقية للوصول الى النتيجة النهائية
   اذا كانت الجملة خاطئة, لا تصححها! فقط اجب ب خاطئ
                    الاجابة النهائية: صحيح او خاطئ

  <</SYS>>
  <s> [INST]
        الجمله: ما أهملت واجبى قط
        الاعراب: ما : اسم إن منصوب وعلامة نصبه الألف
 لنقم بتحليل الجملة خ

response_time,▁▂▂▁█▂▂▁▂█▂▁▂▁▁▂▁▁▂▁▂▁▂▁▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂
response_time_AVG,▁
prompt,[INST]<> ...
response_time,6.69823
response_time_AVG,5.30785


x
T]
 خاطئ

الجملة: قال الطالب: أنا ناجح

التحليل الصحيح:
1. قال: فعل ماضٍ مبني على الفتح
2. الطالب: فاعل مرفوع وعلامة رفعه الضمة الظاهرة على آخره
3. أنا: ضمير منفصل مبني على السكون في محل رفع مبتدأ
4. ناجح: خبر مرفوع وعلامة رفعه الضمة الظاهرة على آخره

إذاً، الإعراب الصحيح للكلمة "قال" هو: فعل ماضٍ. 
1/744
0
x
T]
2/744
 question: <s> [INST] 
  حدد اذا كان الاعراب المعطى للكلمة صحيحه ام خاطئة, فقط اجب بصحيح او خاطئ
  اذا كان الاعراب المعطى خاطئ, اجب ب خاطئ مع تصحيح الاعراب
  
        
        الجمله: قال الطالب : أنا ناجح
        الاعراب: الطالب : فاعل مرفوع وعلامة رفعه الضمة
        لنقم بتحليل الجملة خطوة بخطوة:
        [/INST]
1
x
T]
3/744
1
x
T]
4/744
1
x
T]
5/744
0
x
T]
6/744
1
x
T]
7/744
1
x
T]
8/744
1
x
T]
9/744
1
x
T]
10/744
1
x
T]
11/744
1
x
T]
12/744
1
x
T]
13/744
1
x
T]
14/744
1
x
T]
15/744
0
x
T]
16/744
1
x
T]
17/744
1
x
T]
18/744
0
x
T]
19/744
0
x
T]
20/744
1
x
T]
21/744
1
x
T]
22/744
1
x
T]
23/744
0
x
T]
24/744
1
x
T]
25/744
1
x
T]
26/744
1
x
T]
27/744
1
x
T]
28/744
1
x
T

response_time,▄▁▆▁▁▁▁▁▁▁▁▁▂▁▁▁▁▇▁▁▄▅▃▂▄▄▄▃▅█▃▃█▅▄▃▅▄▃▅
response_time_AVG,▁
prompt,حدد اذا كان الاعر...
response_time,4.28518
response_time_AVG,2.09213
